In [87]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint
import json
from citipy import citipy


from config import gkey

In [9]:
# Setting address to pull city data from WeatherPy
data_address = "../data/cities.csv"

# Creating a DF with the cities.csv info
weather_df = pd.read_csv(data_address)
weather_df = weather_df.drop(columns="Unnamed: 0")
weather_df.head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,chuy,uy,-33.6971,-53.4616,73.26,56,0,12.46
1,new norfolk,au,-42.7826,147.0587,44.80,55,62,1.99
2,hithadhoo,mv,-0.6000,73.0833,80.83,64,0,3.09
3,dillon,us,34.4166,-79.3712,53.87,35,1,19.57
4,saldanha,za,-33.0117,17.9442,69.80,68,0,9.22


In [23]:
# Dropping values with Humidity equal to 100 or over
weather_clean_df = weather_df.loc[weather_df["Humidity"] < 100]
weather_clean_df.head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,chuy,uy,-33.6971,-53.4616,73.26,56,0,12.46
1,new norfolk,au,-42.7826,147.0587,44.80,55,62,1.99
2,hithadhoo,mv,-0.6000,73.0833,80.83,64,0,3.09
3,dillon,us,34.4166,-79.3712,53.87,35,1,19.57
4,saldanha,za,-33.0117,17.9442,69.80,68,0,9.22


In [24]:
# Configure Google API
gmaps.configure(api_key=gkey)

In [25]:
#Setting locations and weight to be called for the heat map and casting them as floats

locations = weather_clean_df[["Lat", "Lng"]].astype(float)

humidity_per = weather_clean_df["Humidity"].astype(float)

In [26]:
# Creating a heat layer using locations and humidity, Max intensity of the heat is the max of weather_df[humidity]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_per,
                                dissipating=False, max_intensity=max(humidity_per),
                                point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Creating Ideal dataframe with the ideal vacation weather conditions
ideal_df = weather_clean_df.loc[(weather_clean_df["Temperature (F)"] > 75) & 
                                (weather_clean_df["Temperature (F)"] < 100) & 
                                (weather_clean_df["Wind Speed (mph)"] < 10) & 
                                (weather_clean_df["Wind Speed (mph)"] > 5) &
                               (weather_clean_df["Cloudiness"] < 5)]
ideal_df = ideal_df.reset_index().drop(columns="index")
ideal_df.head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph)
0,lazaro cardenas,mx,17.9583,-102.2000,85.23,51,0,7.07
1,villa carlos paz,ar,-31.4241,-64.4978,85.98,43,0,7.00
2,mount isa,au,-20.7333,139.5000,82.40,34,0,8.05
3,necochea,ar,-38.5473,-58.7368,87.01,40,0,5.66
4,san luis,ar,-33.2950,-66.3356,87.58,17,0,5.30


In [30]:
# Checking number of cities in Df
print(len(ideal_df["City"]))

13


In [31]:
#Dropping and null values and checking how many were dropped from the previous count
ideal_clean_df = ideal_df.dropna(how="any")
print(len(ideal_clean_df["City"]))

13


In [33]:
# Creating a hotel DF and adding the column Hotel Name, reseting the index

hotel_df = pd.DataFrame({"City": ideal_clean_df["City"],
            "Country": ideal_clean_df["Country"], 
            "Lat": ideal_clean_df["Lat"], 
            "Lng": ideal_clean_df["Lng"], 
            "Temperature (F)": ideal_clean_df["Temperature (F)"], 
            "Humidity": ideal_clean_df["Humidity"], 
            "Cloudiness": ideal_clean_df["Cloudiness"], 
            "Wind Speed (mph)": ideal_clean_df["Wind Speed (mph)"]})

hotel_df["Hotel Name"] = ""

hotel_df.reset_index(drop=True, inplace=True)
hotel_df.head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Hotel Name
0,lazaro cardenas,mx,17.9583,-102.2000,85.23,51,0,7.07,
1,villa carlos paz,ar,-31.4241,-64.4978,85.98,43,0,7.00,
2,mount isa,au,-20.7333,139.5000,82.40,34,0,8.05,
3,necochea,ar,-38.5473,-58.7368,87.01,40,0,5.66,
4,san luis,ar,-33.2950,-66.3356,87.58,17,0,5.30,


In [62]:
locations = hotel_df[["Lat", "Lng"]].astype(str)
print(locations)

         Lat       Lng
0    17.9583    -102.2
1   -31.4241  -64.4978
2   -20.7333     139.5
3   -38.5473  -58.7368
4    -33.295  -66.3356
5     -32.55     115.7
6   -17.7125  -39.2481
7      15.85  -97.0667
8   -24.8667  113.6333
9   -17.3411  -39.2208
10  -32.5661  -53.3758
11  -29.1253  -56.5531
12   16.8634  -99.8901


In [82]:
# Creating variables for the request url based on Google places search by API dictionary. 
# Creating empty lists to append in the for loop

url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
keyword = "hotel"
search_type = "lodging"
radius = "5000"
hotel_lats = []
hotel_lngs = []
hotel_names = []

# Requesting the information from Google API and appending to the lists


for x in range(len(locations)):
    location = f"{locations.iloc[x, 0]}, {locations.iloc[x, 1]}"
    response = requests.get(url + f"location={location}&radius={radius}&type={search_type}&keyword={keyword}&key={gkey}")
    response_json = response.json()
    hotel_lat = response_json["results"][0]["geometry"]["location"]["lat"]
    hotel_lng = response_json["results"][0]["geometry"]["location"]["lng"]
    hotel_name = response_json["results"][0]["name"]
    hotel_lats.append(hotel_lat)
    hotel_lngs.append(hotel_lng)
    hotel_names.append(hotel_name)

In [83]:
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Lat"] = hotel_lats
hotel_df["Hotel Lngs"] = hotel_lngs
hotel_df.head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Hotel Name,Hotel Lat,Hotel Lngs
0,lazaro cardenas,mx,17.9583,-102.2000,85.23,51,0,7.07,City Express Lázaro Cárdenas,17.971610,-102.207043
1,villa carlos paz,ar,-31.4241,-64.4978,85.98,43,0,7.00,Pinares Panorama Suites & Spa,-31.415197,-64.481037
2,mount isa,au,-20.7333,139.5000,82.40,34,0,8.05,ibis Styles Mt Isa Verona,-20.726140,139.492803
3,necochea,ar,-38.5473,-58.7368,87.01,40,0,5.66,Hotel Ñikén Spa & Business Center,-38.581214,-58.733102
4,san luis,ar,-33.2950,-66.3356,87.58,17,0,5.30,Vista Suites Spa & Golf,-33.299476,-66.346451


In [89]:
# hotel_df["Hotel Lat"] = response_json["results"][0]["geometry"]["location"]["lat"]
# hotel_df["Hotel Lng"] = response_json["results"][0]["geometry"]["location"]["lng"]
# hotel_df["Hotel Name"] = response_json["results"][0]["name"]

hotel_lat_lngs = zip(hotel_lats, hotel_lngs)
hotel_cities = []
hotel_countries = []
      
for hotel_lat_lng in hotel_lat_lngs:
    hotel_city = citipy.nearest_city(hotel_lat_lng[0], hotel_lat_lng[1]).city_name
    hotel_country = citipy.nearest_city(hotel_lat_lng[0], hotel_lat_lng[1]).country_code
    hotel_cities.append(hotel_city)
    hotel_countries.append(hotel_country)
print(hotel_cities, hotel_countries)

['lazaro cardenas', 'villa carlos paz', 'mount isa', 'necochea', 'san luis', 'mandurah', 'caravelas', 'puerto escondido', 'carnarvon', 'prado', 'jaguarao', 'itaqui', 'acapulco'] ['mx', 'ar', 'au', 'ar', 'ar', 'au', 'br', 'mx', 'au', 'br', 'br', 'br', 'mx']


In [90]:
hotel_df["Hotel City"] = hotel_cities
hotel_df["Countries"] = hotel_countries
hotel_df.head()

,City,Country,Lat,Lng,Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Hotel Name,Hotel Lat,Hotel Lngs,Hotel City,Countries
0,lazaro cardenas,mx,17.9583,-102.2000,85.23,51,0,7.07,City Express Lázaro Cárdenas,17.971610,-102.207043,lazaro cardenas,mx
1,villa carlos paz,ar,-31.4241,-64.4978,85.98,43,0,7.00,Pinares Panorama Suites & Spa,-31.415197,-64.481037,villa carlos paz,ar
2,mount isa,au,-20.7333,139.5000,82.40,34,0,8.05,ibis Styles Mt Isa Verona,-20.726140,139.492803,mount isa,au
3,necochea,ar,-38.5473,-58.7368,87.01,40,0,5.66,Hotel Ñikén Spa & Business Center,-38.581214,-58.733102,necochea,ar
4,san luis,ar,-33.2950,-66.3356,87.58,17,0,5.30,Vista Suites Spa & Golf,-33.299476,-66.346451,san luis,ar


In [99]:
hotel_locations = hotel_df[["Hotel Lat", "Hotel Lngs"]].astype(float)

hotel_layer = gmaps.symbol_layer(hotel_locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel: {x}\nCity: {y}\nCountry: {z}" for x,y,z in hotel_names hotel_cities hotel_countries])

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

IndexError: list index out of range

In [ ]:
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_)